# Example Injection Test
Below is a run through of a basic, single injection test that can form the core of a larger loop over many trial planets using a parallel architecture

This will use HIP67522 as the example lightcurve, inject a single planet, and try to find it again.

In [1]:
import numpy as np
import glob
import interface as thymeNL
import lcfunctions as lcfunc
import injection_recovery as IJ

Load the data

In [3]:

target_name = 166527623
datafile = 'tessdata/tess2019112060037-s0011-0000000166527623-0143-s_lc.fits'

hip67522 = thymeNL.target(target_name)
hip67522.load_data_tess(datafile)

display(hip67522.data)

rec.array([(1599.95768024, 1.00499221, 1.00096969, 0., 0, 0.),
           (1599.95906914, 1.00411139, 1.0002825 , 0., 0, 0.),
           (1599.96462475, 1.00461384, 1.00080516, 0., 0, 0.), ...,
           (1623.8935936 , 0.99497414, 0.99481859, 0., 0, 0.),
           (1623.89498245, 0.99417769, 0.99401646, 0., 0, 0.),
           (1623.8963713 , 0.99518345, 0.99505984, 0., 0, 0.)],
          dtype=[('t', '<f8'), ('fraw', '<f8'), ('fcor', '<f8'), ('s', '<f8'), ('qual', '<i8'), ('divisions', '<f8')])

Here are the parameters of the planet we will inject and try to recover

In [7]:
per   = 5.0  ## days
rp    = 10.0 ##Rearth, 1.0 here will result in a non-recovery
impact= 0.0  ## Impact parameter < 1.0
ecc   = 0.0  ## Eccentricity, keep it below ~0.9 
t0    = hip67522.data.t[0]+1.0 ##t0
omega = 90.0 ##argument of periastron in degrees.
mstar = 1.22 ## Stellar mass from your own calculations
rstar = 1.38 ## Stellar raius from your own calculations



#### Run the injection-recovery test
This will generate the planet signal, multiply it into the input data, run either notch or locor depending on the input parameters,
and then run BLS (on both the detrended LC and the the BIC statistic if notch was run).
If a planet with matching period/t0 to within 1% in both is recovered at any point, it will stop and output. In future Aaron will add customizable margin for matching here.
###### INPUTS:
    -------
        data (numpy recarray): in the format specified in interface.read_tess_data
        per: The orbital period in days for the model planet
        rp : planet radius in Earth radii
        impact: model impact parameter (0 - 1)
        t0 : time of mid transit, should make sense relative to your input time array.
        ecc: model eccentricity for the planet. Big numbers can cause numerical problems (best below 0.95).
        omega: model argument of periastron passage angle for the planet in degrees
        mstar: mass of the host star in Solar units
        rstar: radius of the host star in Solar units
        
###### OPTIONALS:
    ----------
    windowsize=1.0: if demode=1 this is the nootch window size, if demode=2, this is the locor star rotation period.
    alias_num=2.0: as per locors alias_num,. the minimum period to alias the rotation period of the stars (windowsize) to.
    demode=1: 1=notch, 2=locor
    min_period=1.0: minimum bls search period in days
    max_period=15.0: maximum bls search period in days
    forcenull=False: If true, always will reject the notch model, for testing purposes.
    exposuretime=None: The exposure time in minutes of your input data. If None, will compute the most common time between data points and use that 
    ldpars=[0.4,0.3]: quadratic limb darkening parameters to use for star model
    oversample=20: oversample rate for batman, 20 is usually ok
    
###### OUTPUTS:
    --------
    detected_it: 1 if recovered, 0 if not
    detp: recovered period if recovered
    dett0: recovered T0 if recovered
    detdp: recovered depth if recovered (not trustable, and meaningless if BIC search was run).
    '''

In [8]:
detected_it,detp,dett0,detdp = IJ.inject_recover_tess(hip67522.data,per,rp,impact,t0,ecc,omega,mstar,rstar,windowsize=0.5,demode=1,alias_num=2.0,min_period=1.00001,max_period=15.0,forcenull=False,exposuretime=None,ldpars=[0.4,0.3],oversample=20)

if detected_it == 1:
    print('Planet recovered :)!')
else: print('Planet not Recovered :(!')




  0%|          | 10/13883 [00:00<02:35, 89.30it/s]

Running Detrend


100%|██████████| 13883/13883 [02:52<00:00, 80.66it/s] 


Doing BIC Mode
skipping gomask search
Planet not Recovered :(!


### Scaling up
For a script that runs an injection recovery using multiproccessing, see tess_injrec_mpi.py <br>
This will require installing mpi4py and having a working version of MPI (e.g., openmpi) installed on your system<br>


### Plot the results
Here is how to plot the results from a previously generated large test using tess_injrec_mpi.py
This example is for Ben Tofflemires Octans planet (HD 110082)


In [2]:
files = glob.glob('./tessinjrec/Octans_planet/*.pkl')
IJ.plot_ijresults(files,targetname = 'HD110082',outdir = './',detected_planets = [[10.18,3.3]],extratag = 'test',numbins=15,oldfile=True)


DONE


### This Saved he plots to your working directory, go check them out!!